In [1]:
import json
import tensorflow as tf
import numpy as np
import h5py
import copy
import matplotlib.pyplot as plt
import matplotlib.path as mplPath

from fr3D.train.utils import setup_datasets
from fr3D.data.utils import get_normalization_type
from fr3D.models import ConvAutoencoder, ConvAutoencoderCGAN, ConvAutoencoderC
from fr3D.utils import mape_with_threshold

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
Re = 500
expt_variables = ['Pressure']

dataset_path = f'/fr3D/postprocessed/annulus_64_plane.h5'

#ConvAutoencoderC
experiment_configs = {'Pressure':f'/fr3D/configs/training/ConvAutoencoderC_PIV.json'}
weights_paths = {'Pressure':f'/storage/weights500Old/ConvAutoencoderC_PIV_Annulus64/ConvAutoencoderC_PIV_Annulus64.h5'}



datasetf = h5py.File(dataset_path,'r')

shuf_buf = 1

train_datasets = {}
test_datasets = {}
sensor_shapes = {}
full_field_shapes = {}
normalizers = {}

for expt_variable in experiment_configs:
    config = json.load(open(experiment_configs[expt_variable],'r'))
    train_datasets[expt_variable], test_datasets[expt_variable] = setup_datasets(config, dataset_path, shuf_buf, case_names=True, evaluation=True)
    sensor_shapes[expt_variable] = train_datasets[expt_variable].element_spec[0][0].shape
    full_field_shapes[expt_variable] = train_datasets[expt_variable].element_spec[0][1].shape
    normalizers[expt_variable] = get_normalization_type(config['dataset']['node_configurations'])

Training geometries: ('shape_69', 'shape_329', 'shape_46', 'shape_241', 'shape_239', 'shape_228', 'shape_163', 'shape_63', 'shape_137', 'shape_102', 'shape_157', 'shape_197', 'shape_52', 'shape_213', 'shape_84', 'shape_113', 'shape_58', 'shape_59', 'shape_146', 'shape_206', 'shape_331', 'shape_8', 'shape_18', 'shape_236', 'shape_43', 'shape_15', 'shape_23', 'shape_92', 'shape_135', 'shape_50', 'shape_14', 'shape_255', 'shape_75', 'shape_183', 'shape_89', 'shape_12', 'shape_374', 'shape_37', 'shape_218', 'shape_150', 'shape_31', 'shape_216', 'shape_13', 'shape_90', 'shape_182', 'shape_159', 'shape_67', 'shape_170', 'shape_358', 'shape_147', 'shape_17', 'shape_110', 'shape_152', 'shape_224', 'shape_130', 'shape_60', 'shape_126', 'shape_369', 'shape_22', 'shape_83', 'shape_338', 'shape_97', 'shape_33', 'shape_237', 'shape_325', 'shape_39', 'shape_244', 'shape_41', 'shape_355', 'shape_94', 'shape_61', 'shape_234', 'shape_34', 'shape_29', 'shape_95', 'shape_339', 'shape_55', 'shape_21', 'sh

2023-01-05 23:54:38.693450: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2023-01-05 23:54:38.693619: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA
2023-01-05 23:54:38.850506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 23:54:39.315850: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37699 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:43:00.0, compute capability: 8.0


In [3]:
models = {}

for v in expt_variables:
    config = json.load(open(experiment_configs[v],'r'))
    model = ConvAutoencoderC(dense_input_units=sensor_shapes[v][1],
                             autoencoder_input_shape=full_field_shapes[v][1:],
                             **config['model'])
    loss_fn = "mse"#tf.keras.losses.get(config['training']['loss'])
    model.compile(l_optimizer= tf.keras.optimizers.get(config['training']['l_optimizer']),
                  loss=loss_fn,
                  optimizer = tf.keras.optimizers.get(config['training']['ae_optimizer']),
                  metrics = config['training'].get('metrics', None))
    model.load_weights(weights_paths[v])
    models[v] = model

In [5]:
@tf.function
def get_predictions(model, inp, target):
    return model(inp, autoencode=False), model(target, autoencode=True)

def undo_normalization(normalizer, norm_param, pred, ae_pred, target):
    ntarget = normalizer.undo(target, norm_param[:,0,:])
    npred = normalizer.undo(pred, norm_param[:,0,:])
    nae_pred = normalizer.undo(ae_pred, norm_param[:,0,:])
    return npred, nae_pred, ntarget

@tf.function
def check_in_box(coords, bounds_lower, bounds_upper):
    newshape = tf.concat([tf.ones((tf.rank(coords)-1,), dtype=tf.int32), tf.shape(bounds_lower)[:1]],0)
    bounds_lower_r = tf.reshape(bounds_lower, newshape)
    bounds_upper_r = tf.reshape(bounds_upper, newshape)
    lower_cond = tf.reduce_all(coords > bounds_lower, axis=-1)
    upper_cond = tf.reduce_all(coords < bounds_upper, axis=-1)
    return tf.logical_and(lower_cond, upper_cond)

def compute_metrics(coords, v, mapes, umapes, mses, umses, pred, target, npred, ntarget):
    lower_box_bounds = tf.constant([-3.0,-3.0,-0.1])
    upper_box_bounds = tf.constant([8.0,3.0,10.1])
    in_box = check_in_box(coords, lower_box_bounds, upper_box_bounds)
    in_box_r = tf.reshape(in_box, tf.concat([tf.shape(in_box), tf.ones((tf.rank(p_npred) - tf.rank(in_box),), tf.int32)], 0))
    in_box_rf = tf.cast(in_box_r, tf.float32)
    
    _lf =tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    mapes[v].append(
        mape_with_threshold(npred, ntarget, pcterror_threshold=100.0, max_magnitude_threshold=0.03, sample_weights=in_box_rf, axis=tf.range(1, tf.rank(in_box_rf)))
    )
    umapes[v].append(
        mape_with_threshold(pred, target, pcterror_threshold=100.0, max_magnitude_threshold=0.03, sample_weights=in_box_rf, axis=tf.range(1, tf.rank(in_box_rf)))
    )
    mses[v].append(tf.reduce_mean(_lf(npred, ntarget, in_box_rf), axis=tf.range(1, tf.rank(in_box_rf)-1)))
    umses[v].append(tf.reduce_mean(_lf(pred, target, in_box_rf), axis=tf.range(1, tf.rank(in_box_rf)-1)))


dataset_iterators = {v: iter(test_datasets[v]) for v in test_datasets}

pred_forces = []
ae_pred_forces = []
target_forces = []

pred_mapes = {k:[] for k in dataset_iterators.keys()}
pred_unnormalized_mapes = {k:[] for k in dataset_iterators.keys()}
pred_mses = {k:[] for k in dataset_iterators.keys()}
pred_unnormalized_mses = {k:[] for k in dataset_iterators.keys()}

ae_pred_mapes = {k:[] for k in dataset_iterators.keys()}
ae_pred_unnormalized_mapes = {k:[] for k in dataset_iterators.keys()}
ae_pred_mses = {k:[] for k in dataset_iterators.keys()}
ae_pred_unnormalized_mses = {k:[] for k in dataset_iterators.keys()}

for pdata in zip(*dataset_iterators.values()):
    (p_inp, p_target, p_norm_param), case_name = pdata[0]
    
    coords = tf.convert_to_tensor(np.stack([datasetf[c.decode()]['full_field_coords'] for c in case_name.numpy()],0))
    
    p_pred, p_ae_pred = get_predictions(models['Pressure'], p_inp, p_target)
    p_npred, p_nae_pred, p_ntarget = undo_normalization(normalizers['Pressure'], p_norm_param, p_pred, p_ae_pred, p_target)
   
    pred_pressures = p_npred[...,0]
    ae_pred_pressures = p_nae_pred[...,0]
    target_pressures = p_ntarget[...,0]
    
    compute_metrics(coords, 'Pressure', pred_mapes, pred_unnormalized_mapes, pred_mses, pred_unnormalized_mses,
                       p_pred, p_target, p_npred, p_ntarget)
    
    compute_metrics(coords, 'Pressure', ae_pred_mapes, ae_pred_unnormalized_mapes, ae_pred_mses, ae_pred_unnormalized_mses,
                       p_ae_pred, p_target, p_nae_pred, p_ntarget)

In [6]:
pred_mapes = {k:tf.concat(pred_mapes[k],0) for k in dataset_iterators.keys()}
pred_unnormalized_mapes = {k:tf.concat(pred_unnormalized_mapes[k],0) for k in dataset_iterators.keys()}
pred_mses = {k:tf.concat(pred_mses[k],0) for k in dataset_iterators.keys()}
pred_unnormalized_mses = {k:tf.concat(pred_unnormalized_mses[k],0) for k in dataset_iterators.keys()}

ae_pred_mapes = {k:tf.concat(ae_pred_mapes[k],0) for k in dataset_iterators.keys()}
ae_pred_unnormalized_mapes = {k:tf.concat(ae_pred_unnormalized_mapes[k],0) for k in dataset_iterators.keys()}
ae_pred_mses = {k:tf.concat(ae_pred_mses[k],0) for k in dataset_iterators.keys()}
ae_pred_unnormalized_mses = {k:tf.concat(ae_pred_unnormalized_mses[k],0) for k in dataset_iterators.keys()}


In [7]:
def print_metrics(v, s, mapes, unnormalized_mapes, mses, unnormalized_mses):
    print(f">> {v} {s}")
    print(f'MAPE: {tf.reduce_mean(mapes[v]).numpy()}')
    print(f'Unnormalized MAPE: {tf.reduce_mean(unnormalized_mapes[v]).numpy()}')
    print(f'MSE: {tf.reduce_mean(mses[v]).numpy()}')
    print(f'Unnormalized MSE: {tf.reduce_mean(unnormalized_mses[v]).numpy()}')
    print('')

print('===Overall metrics===')

print_metrics('Pressure', 'Pred', pred_mapes, pred_unnormalized_mapes, pred_mses, pred_unnormalized_mses)
print_metrics('Pressure', 'AE Pred', ae_pred_mapes, ae_pred_unnormalized_mapes, ae_pred_mses, ae_pred_unnormalized_mses)


===Overall metrics===
>> Pressure Pred
MAPE: 24.76392364501953
Unnormalized MAPE: 17.91895294189453
MSE: 0.3148247301578522
Unnormalized MSE: 0.006620199419558048

>> Pressure AE Pred
MAPE: 13.87522029876709
Unnormalized MAPE: 8.794646263122559
MSE: 0.0076378704980015755
Unnormalized MSE: 0.0007551303715445101

